# VOC Exploration Example
<div align="center">

  <a href="https://ultralytics.com/yolo" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov8/banner-yolov8.png"></a>

  [中文](https://docs.ultralytics.com/zh/) | [한국어](https://docs.ultralytics.com/ko/) | [日本語](https://docs.ultralytics.com/ja/) | [Русский](https://docs.ultralytics.com/ru/) | [Deutsch](https://docs.ultralytics.com/de/) | [Français](https://docs.ultralytics.com/fr/) | [Español](https://docs.ultralytics.com/es/) | [Português](https://docs.ultralytics.com/pt/) | [Türkçe](https://docs.ultralytics.com/tr/) | [Tiếng Việt](https://docs.ultralytics.com/vi/) | [العربية](https://docs.ultralytics.com/ar/)

  <a href="https://console.paperspace.com/github/ultralytics/ultralytics"><img src="https://assets.paperspace.io/img/gradient-badge.svg" alt="Run on Gradient"/></a>
  <a href="https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://www.kaggle.com/models/ultralytics/yolo11"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>

Welcome to the Ultralytics Explorer API notebook!  This notebook serves as the starting point for exploring the various resources available to help you get started with using Ultralytics to explore your datasets using with the power of semantic search. You can utilities out of the box that allow you to examine specific types of labels using vector search or even SQL queries.

We hope that the resources in this notebook will help you get the most out of Ultralytics. Please browse the Explorer <a href="https://docs.ultralytics.com/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/ultralytics">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!

Try `yolo explorer` powered by Exlorer API

Simply `pip install ultralytics` and run `yolo explorer` in your terminal to run custom queries and semantic search on your datasets right inside your browser!

</div>

## Ultralytics Explorer support deprecated ⚠️

As of **`ultralytics>=8.3.10`**, Ultralytics explorer support has been deprecated. But don’t worry! You can now access similar and even enhanced functionality through [Ultralytics HUB](https://hub.ultralytics.com/), our intuitive no-code platform designed to streamline your workflow. With Ultralytics HUB, you can continue exploring, visualizing, and managing your data effortlessly, all without writing a single line of code. Make sure to check it out and take advantage of its powerful features!🚀

## Setup
Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

In [0]:
%pip install ultralytics[explorer] openai
import ultralytics

ultralytics.checks()

In [0]:
from ultralytics import Explorer

## Similarity search
Utilize the power of vector similarity search to find the similar data points in your dataset along with their distance in the embedding space. Simply create an embeddings table for the given dataset-model pair. It is only needed once and it is reused automatically.


In [0]:
exp = Explorer("VOC.yaml", model="yolo11n.pt")
exp.create_embeddings_table()

One the embeddings table is built, you can get run semantic search in any of the following ways:
- On a given index / list of indices in the dataset like - `exp.get_similar(idx=[1,10], limit=10)`
- On any image/ list of images not in the dataset  - `exp.get_similar(img=["path/to/img1", "path/to/img2"], limit=10)`
In case of multiple inputs, the aggregade of their embeddings is used.

You get a pandas dataframe with the `limit` number of most similar data points to the input, along with their distance in the embedding space. You can use this dataset to perform further filtering
<img width="1120" alt="Screenshot 2024-01-06 at 9 45 42 PM" src="https://github.com/AyushExel/assets/assets/15766192/7742ac57-e22a-4cea-a0f9-2b2a257483c5">


In [0]:
similar = exp.get_similar(idx=1, limit=10)
similar.head()

You can use the also plot the similar samples directly using the `plot_similar` util
<p>

 <img src="https://github.com/AyushExel/assets/assets/15766192/a3c9247b-9271-47df-aaa5-36d96c5034b1" />
</p>


In [0]:
exp.plot_similar(idx=6500, limit=20)
# exp.plot_similar(idx=[100,101], limit=10) # Can also pass list of idxs or imgs

In [0]:
exp.plot_similar(
    img="https://ultralytics.com/images/bus.jpg", limit=10, labels=False
)  # Can also pass any external images

<p>
<img  src="https://github.com/AyushExel/assets/assets/15766192/8e011195-b0da-43ef-b3cd-5fb6f383037e">

</p>

## 2. Ask AI: Search or filter with Natural Language
You can prompt the Explorer object with the kind of data points you want to see and it'll try to return a dataframe with those. Because it is powered by LLMs, it doesn't always get it right. In that case, it'll return None.
<p>
<img width="1131" alt="Screenshot 2024-01-07 at 2 34 53 PM" src="https://github.com/AyushExel/assets/assets/15766192/c4a69fd9-e54f-4d6a-aba5-dc9cfae1bc04">

</p>



In [0]:
df = exp.ask_ai("show me images containing more than 10 objects with at least 2 persons")
df.head(5)

for plotting these results you can use `plot_query_result` util
Example:
```
plt = plot_query_result(exp.ask_ai("show me 10 images containing exactly 2 persons"))
Image.fromarray(plt)
```
<p>
    <img src="https://github.com/AyushExel/assets/assets/15766192/2cb780de-d05b-4412-a526-7f7f0f10e669">

</p>

In [0]:
# plot
from PIL import Image

from ultralytics.data.explorer import plot_query_result

plt = plot_query_result(exp.ask_ai("show me 10 images containing exactly 2 persons"))
Image.fromarray(plt)

## 3. Run SQL queries on your Dataset!
Sometimes you might want to investigate a certain type of entries in your dataset. For this Explorer allows you to execute SQL queries.
It accepts either of the formats:
- Queries beginning with "WHERE" will automatically select all columns. This can be thought of as a short-hand query
- You can also write full queries where you can specify which columns to select

This can be used to investigate model performance and specific data points. For example:
- let's say your model struggles on images that have humans and dogs. You can write a query like this to select the points that have at least 2 humans AND at least one dog.

You can combine SQL query and semantic search to filter down to specific type of results
<img width="994" alt="Screenshot 2024-01-06 at 9 47 30 PM" src="https://github.com/AyushExel/assets/assets/15766192/92bc3178-c151-4cd5-8007-c76178deb113">


In [0]:
table = exp.sql_query("WHERE labels LIKE '%person, person%' AND labels LIKE '%dog%' LIMIT 10")
table

Just like similarity search, you also get a util to directly plot the sql queries using `exp.plot_sql_query`
<img src="https://github.com/AyushExel/assets/assets/15766192/f8b66629-8dd0-419e-8f44-9837969ba678">


In [0]:
exp.plot_sql_query("WHERE labels LIKE '%person, person%' AND labels LIKE '%dog%' LIMIT 10", labels=True)

## 3. Working with embeddings Table (Advanced)
Explorer works on [LanceDB](https://lancedb.github.io/lancedb/) tables internally. You can access this table directly, using `Explorer.table` object and run raw queries, push down pre and post filters, etc.

In [0]:
table = exp.table
table.schema

### Run raw queries
Vector Search finds the nearest vectors from the database. In a recommendation system or search engine, you can find similar products from the one you searched. In LLM and other AI applications, each data point can be presented by the embeddings generated from some models, it returns the most relevant features.

A search in high-dimensional vector space, is to find K-Nearest-Neighbors (KNN) of the query vector.

Metric
In LanceDB, a Metric is the way to describe the distance between a pair of vectors. Currently, it supports the following metrics:
- L2
- Cosine
- Dot
Explorer's similarity search uses L2 by default. You can run queries on tables directly, or use the lance format to build custom utilities to manage datasets. More details on available LanceDB table ops in the [docs](https://lancedb.github.io/lancedb/)

<img width="1015" alt="Screenshot 2024-01-06 at 9 48 35 PM" src="https://github.com/AyushExel/assets/assets/15766192/a2ccdaf3-8877-4f70-bf47-8a9bd2bb20c0">


In [0]:
dummy_img_embedding = [i for i in range(256)]
table.search(dummy_img_embedding).limit(5).to_pandas()

### Inter-conversion to popular data formats

In [0]:
df = table.to_pandas()
pa_table = table.to_arrow()

### Work with Embeddings
You can access the raw embedding from lancedb Table and analyse it. The image embeddings are stored in column `vector`

In [0]:
import numpy as np

embeddings = table.to_pandas()["vector"].tolist()
embeddings = np.array(embeddings)

### Scatterplot
One of the preliminary steps in analysing embeddings is by plotting them in 2D space via dimensionality reduction. Let's try an example

<img width="646" alt="Screenshot 2024-01-06 at 9 48 58 PM" src="https://github.com/AyushExel/assets/assets/15766192/9e1da25c-face-4426-abc0-2f64a4e4952c">


In [0]:
!pip install scikit-learn --q

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

# Reduce dimensions using PCA to 3 components for visualization in 3D
pca = PCA(n_components=3)
reduced_data = pca.fit_transform(embeddings)

# Create a 3D scatter plot using Matplotlib's Axes3D
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

# Scatter plot
ax.scatter(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], alpha=0.5)
ax.set_title("3D Scatter Plot of Reduced 256-Dimensional Data (PCA)")
ax.set_xlabel("Component 1")
ax.set_ylabel("Component 2")
ax.set_zlabel("Component 3")

plt.show()

## 4. Similarity Index
Here's a simple example of an operation powered by the embeddings table. Explorer comes with a `similarity_index` operation-
* It tries to estimate how similar each data point is with the rest of the dataset.
*  It does that by counting how many image embeddings lie closer than `max_dist` to the current image in the generated embedding space, considering `top_k` similar images at a time.

For a given dataset, model, `max_dist` & `top_k` the similarity index once generated will be reused. In case, your dataset has changed, or you simply need to regenerate the similarity index, you can pass `force=True`.
Similar to vector and SQL search, this also comes with a util to directly plot it. Let's look at the plot first
<img width="633" alt="Screenshot 2024-01-06 at 9 49 36 PM" src="https://github.com/AyushExel/assets/assets/15766192/96a9d984-4a72-4784-ace1-428676ee2bdd">



In [0]:
exp.plot_similarity_index(max_dist=0.2, top_k=0.01)

Now let's look at the output of the operation

In [0]:
import numpy as np

sim_idx = exp.similarity_index(max_dist=0.2, top_k=0.01, force=False)

In [0]:
sim_idx

Let's create a query to see what data points have similarity count of more than 30 and plot images similar to them.

In [0]:
import numpy as np

sim_count = np.array(sim_idx["count"])
sim_idx["im_file"][sim_count > 30]

You should see something like this
<img src="https://github.com/AyushExel/assets/assets/15766192/649bc366-ca2d-46ea-bfd9-3097cf575584">


In [0]:
exp.plot_similar(idx=[7146, 14035])  # Using avg embeddings of 2 images